# End to End Agent

In [ ]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Bob and I live in SF.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob and I live in SF.
================================== Ai Message ==================================

Hi Bob, nice to meet you too. Is there anything I can help you with?


---
# Setup

##### LangChain Installation

In [1]:
# Install LangChain
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 11.6 MB/s eta 0:00:00


##### LangSmith

In [ ]:
from google.colab import userdata
import os
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGSMITH_API_KEY"] = userdata.get('Smith2')

##### Tavily

In [ ]:
from google.colab import userdata
import os
os.environ["TAVILY_API_KEY"] = userdata.get('tavily')

---
# Define Tools

In [ ]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco, CA', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1750776218, 'localtime': '2025-06-24 07:43'}, 'current': {'last_updated_epoch': 1750775400, 'last_updated': '2025-06-24 07:30', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 7.8, 'wind_kph': 12.6, 'wind_degree': 233, 'wind_dir': 'SW', 'pressure_mb': 1015.0, 'pressure_in': 29.98, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'cloud': 100, 'feelslike_c': 11.6, 'feelslike_f': 52.9, 'windchill_c': 8.6, 'windchill_f': 47.5, 'heatindex_c': 10.2, 'heatindex_f': 50.4, 'dewpoint_c': 10.5, 'de

---
# Using Language Model

Setup model

In [2]:
!pip install -qU "langchain[google-genai]"

from google.colab import userdata

# Retrieve the Gemini API key from Colab's user data
gemini_api_key = userdata.get('gemini')

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai",google_api_key=gemini_api_key)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [3]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hi there! How can I help you today?'

enable tool binding

In [ ]:
model_with_tools = model.bind_tools(tools)

see changes in output

In [ ]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hi there! How can I help you today?

Tool calls: []


no content ; no tool call

In [ ]:
query = "Search for the weather in Bangalore"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in Bangalore'}, 'id': 'd6b294a3-a839-4012-bda1-33d80d0bd4d8', 'type': 'tool_call'}]


---
# Create the Agent

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

---
# Run the Agent

example 1 - No need for tool call Query

In [ ]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hi there! How can I help you today?


example 2 -  requires Tool Call

In [ ]:
input_message = {"role": "user", "content": "Whats happening in Iran. Add details"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Whats happening in Iran. Add details
================================== Ai Message ==================================
Tool Calls:
  tavily_search (272b9ecf-4637-476f-854d-062ce45f6134)
 Call ID: 272b9ecf-4637-476f-854d-062ce45f6134
  Args:
    search_depth: advanced
    query: What's happening in Iran
================================= Tool Message =================================
Name: tavily_search

{"query": "What's happening in Iran", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.amnesty.org/en/location/middle-east-and-north-africa/middle-east/iran/report-iran/", "title": "Human rights in Iran - Amnesty International", "content": "Authorities failed to address Iran’s environmental crisis, marked by loss of lakes, rivers and wetlands; groundwater depletion; deforestation; water pollution from discharge of wastewater into urban water sources; land subsidence

! Error 🔥
For this* Query it said It cannot access internet.
*"Search for the weather in SF with some detail"

---
# Streaming Messages

To show intermediate progress, we can stream back messages as they occur

In [ ]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF with some detail
================================== Ai Message ==================================
Tool Calls:
  tavily_search (26b638c8-b4cc-45f3-ab13-499268ddbe1e)
 Call ID: 26b638c8-b4cc-45f3-ab13-499268ddbe1e
  Args:
    search_depth: advanced
    query: weather in San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/june/2025-06-24", "title": "Tuesday, June 24, 2025. San Francisco, CA - Weather Forecast", "content": "San Francisco, California Weather: \n\nTuesday, June 24, 2025\n\nDay 61°\n\nNight 54°\n\nPrecipitation 0 %\n\nWind 14 mph\n\nUV Index (0 - 11+)10\n\n[Wednesday](https://weathershogun.com/weather/usa/ca/san-francisco/480/june

---
# Streaming Tokens

In [ ]:
#We can do this by specifying stream_mode="messages".
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The weather in San| Francisco, California for Tuesday, June 24, 2025| will be: Day 61°F, Night 54°F,| Precipitation 0%, Wind 14 mph, UV Index 10.|

---
# Adding in memory

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
#Agent is Model, tools and memory
agent_executor = create_react_agent(model, tools, checkpointer=memory)


config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
#check if remembers? - 1
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hi Bob! How can I help you today?


In [ ]:
#check if remembers? - 2
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob.


In [ ]:
#change thread id for new conversation
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I do not have access to personally identifiable information. That includes your name. I'm designed to protect your privacy.


##### Rough Play Issue Flag ⛳

❗ Notice that 10 asked but only few provided? Why?

In [ ]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's top ten news of the day?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's top ten news of the day?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (234cab1c-cf70-41f6-be75-01ec8b3b7238)
 Call ID: 234cab1c-cf70-41f6-be75-01ec8b3b7238
  Args:
    query: Top ten news of the day
    topic: news
================================= Tool Message =================================
Name: tavily_search

{"query": "Top ten news of the day", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.euronews.com/2025/06/20/new-missile-enters-israel-iran-conflict-what-we-know-about-tehrans-sejil", "title": "New missile in Israel-Iran conflict: What we know about Tehran's Sejil - Euronews.com", "score": 0.4119392, "published_date": "Fri, 20 Jun 2025 11:51:44 GMT", "content": "Europe NewsImage 3: Palestinians carry sacks and boxes of food and humanitarian aid that was unloaded from a World Food P

❗ Notice how it snowballs coz persistent memory loops in the earlier response of AI. Where it is getting confused and mentioning it. It is a big problem from a memory usefulness perspective.

In [ ]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What was the second news item in your response?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What was the second news item in your response?
================================== Ai Message ==================================

You are absolutely right to keep me on track. I apologize for the repeated errors.

When I provided the "top ten news of the day," the second news item I listed was: "Dangerous heat waves, box office bounces back, extreme day trips: Catch up on the day’s stories - CNN"


---
---
# *Build-Agent Notes*

```
Steps :
--> Define Tools
--> Define LLM
--> build agent
--> add memory

```



### **Model * tool * Memory = Agent**

#### **Issue Flag**
Refer "Rough Play" and **Notice** how **AIs confusion snowballs** coz persistent memory loops in the earlier response of AI. Where it is getting confused and mentioning it. It is a big problem from a memory usefulness perspective.


